In [ ]:
#Installation

# Clone the ibm/tsfm
#! git clone https://github.com/ibm-granite/granite-tsfm.git
#! ls

# Change directory. Move inside the tsfm repo.
#%cd granite-tsfm
#! ls

# Relax requirement for python version < 3.12
#! sed -i.orig 's/3\.12/3.13/g' pyproject.toml

# Install the tsfm library
#! pip install ".[notebooks]"
#! python3 -m pip install ".[notebooks]"
#! pip3 install ".[notebooks]"

NO SE SELECCIONA PORQUE LA LONGITUD DE LA SECUENCIA DE ENTRADA ESTÁ PREFIJADA A DOS VALORES => NO HAY SUFICIENTE FLEXIBILIDAD.

In [32]:
import os

In [37]:
os.chdir(os.path.expanduser("~/work/nbs_pipeline"))

In [1]:
#| export
verbose                       = 0
check_memory_usage            = True
time_flag                     = True
window_size_percentage        = True
show_plots                    = True
reset_kernel                  = True
pre_configured_case           = True
case_id                       = 7
frequency_factor              = 1
frequency_factor_change_alias = True
check_parameters              = True
cuda_device                   = 0
remove_lambdas_flag           = True

In [2]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download
import sys
import dvats.utils as ut
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

import uni2ts
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_next_multi
import pyarrow.feather as ft
from gluonts.transform.split import TFTInstanceSplitter
from gluonts.transform.sampler import TestSplitSampler
import numpy as np
import einops
import torch.nn.functional as F
from dvats.memory import gpu_memory_status
import dvats.config as cfg_
import warnings
warnings.filterwarnings("ignore", module="umap")
import os
import sys
sys.path.append(os.path.abspath('..'))
from dvats.all import *
from fastcore.all import *
from tsai.basics import *
from tsai.models.InceptionTimePlus import *
from tsai.callback.MVP import *
import matplotlib.colors as colors
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback
import wandb

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedInputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedOutputStream size changed, may indicate binary 


Octave is ready <oct2py.core.Oct2Py object at 0x7fa8ec748ee0>

















In [5]:
import tsfm_public

In [15]:
tiny_mixer_for_prediction = tsfm_public.TinyTimeMixerForPrediction.from_pretrained(
            "ibm/TTM", revision="1024_96_v1"
        )

In [16]:
enc_learner = tiny_mixer_for_prediction.backbone

In [17]:
enc_learner.named_modules

<bound method Module.named_modules of TinyTimeMixerModel(
  (encoder): TinyTimeMixerEncoder(
    (patcher): Linear(in_features=64, out_features=192, bias=True)
    (mlp_mixer_encoder): TinyTimeMixerBlock(
      (mixers): ModuleList(
        (0): TinyTimeMixerAdaptivePatchingBlock(
          (mixer_layers): ModuleList(
            (0-1): 2 x TinyTimeMixerLayer(
              (patch_mixer): PatchMixerBlock(
                (norm): TinyTimeMixerNormLayer(
                  (norm): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
                )
                (mlp): TinyTimeMixerMLP(
                  (fc1): Linear(in_features=64, out_features=128, bias=True)
                  (dropout1): Dropout(p=0.2, inplace=False)
                  (fc2): Linear(in_features=128, out_features=64, bias=True)
                  (dropout2): Dropout(p=0.2, inplace=False)
                )
                (gating_block): TinyTimeMixerGatedAttention(
                  (attn_layer): Linear(in_features=6

### Averiguando la capa interesante del modelo 

In [30]:
module = enc_learner.encoder.mlp_mixer_encoder.mixers[2].mixer_layers[1].feature_mixer.gating_block.attn_softmax
module.named_modules

<bound method Module.named_modules of Softmax(dim=-1)>

### Construyendo el dataset

In [38]:
#| export
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)
if pre_configured_case: 
    cfg_.force_artifact_config_mvp(
        config = config,
        id = case_id,
        verbose = verbose, 
        both = verbose > 0,
        frequency_factor = frequency_factor,
        frequency_factor_change_alias = frequency_factor_change_alias
    )
#| export
path = os.path.expanduser("~/work/nbs_pipeline/")
name="02c_encoder_moment-embedding"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
if verbose > 0: print("runname: "+runname)
if verbose > 0: cfg_.show_attrdict(config)

#| export
if verbose > 0: print("--> Wandb init")
run = wandb.init(
    entity = user,
    # work-nbs is a place to log draft runs
    project=project,
    group=config.wandb_group,
    job_type=job_type,
    allow_val_change=True,
    mode=config.analysis_mode,
    config=config,
    # When use_wandb is false the run is not linked to a personal account
    #NOTE: This is not working right now
    anonymous = 'never' if config.use_wandb else 'must', 
    resume=False,
    name = runname
)
if verbose > 0: print("Wandb init -->")
config = run.config  # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
#| export
config = run.config  # Object for storing hyperparameters
if verbose > 0: cfg_.show_attrdict(config)
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
train_artifact = artifacts_gettr(config.train_artifact)
if verbose > 0: print("---> W&B Train Artifact")

#| export
import pyarrow.feather as ft
df_train = train_artifact.to_df()
print("df_train ~", df_train.shape)
print(df_train.shape)
display(df_train.head())
#| export
if verbose > 0: print("---> Sliding window | ", config.w,  " | ", config.stride )
sw = SlidingWindow(window_len=config.w, stride=config.stride, get_y=[])
if verbose > 0: print(" Sliding window | ", config.w,  " | ", config.stride, "---> | df_train ~ ", df_train.shape )
X_train, _ = sw(df_train)
if verbose > 0: print(" sw_df_train | ", config.w,  " | ", config.stride, "--->" )
print("X_train ~", X_train.shape)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /home/macu/work/nbs_pipeline/02c_encoder_moment-embedding.ipynb.
wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


df_train ~ (550, 3)
(550, 3)


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


X_train ~ (521, 3, 30)


### Adaptando la llamada al modelo

In [41]:
? enc_learner.forward

Signature:
 enc_learner.forward(
    past_values: torch.Tensor,
    past_observed_mask: Optional[torch.Tensor] = None,
    output_hidden_states: Optional[bool] = False,
    return_dict: Optional[bool] = None,
    freq_token: Optional[torch.Tensor] = None,
) -> tsfm_public.models.tinytimemixer.modeling_tinytimemixer.TinyTimeMixerModelOutput
Docstring:
The [`TinyTimeMixerModel`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    past_values (`torch.FloatTensor` of shape `(batch_size, seq_length, num_input_channels)`):
        Context values of the time series. For a forecasting task, this denotes the history/past time series values.
        For univariate time series, `num_input_channels` dimension sh

In [42]:
X_train.shape

(521, 3, 30)

In [44]:
past_values = einops.rearrange(
    torch.from_numpy(X_train),
    "num_windows num_vars window_len -> num_windows window_len num_vars"
)

In [ ]:
module = enc_learner.encoder.mlp_mixer_encoder.mixers[2].mixer_layers[1].feature_mixer.gating_block.attn_softmax
module.named_modules

In [46]:
acts = get_acts(
    model = enc_learner,
    module = module,
    cpu = False,
    past_values = past_values
)

ValueError: Input sequence length (30) doesn't match model configuration (1024).